<img align="right" src="images/tf.png" width="128"/>
<img align="right" src="images/logo.png" width="128"/>
<img align="right" src="images/dans.png" width="128"/>

---

To get started: consult [start](start.ipynb)

---

# Display

We show the ins and outs of displaying cuneiform ATF transcriptions.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tf.app import use

In [3]:
A = use("Nino-cunei/ninmed", hoist=globals())

This is Text-Fabric 9.2.5
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

45 features found and 0 ignored


We pick an example face with which we illustrate many ways to represent cuneiform text.

In [4]:
exampleFace = ("P285109", "reverse")
f = T.nodeFromSection(exampleFace)
lines = L.d(f, otype="line")

# Raw text

The most basic way is to show the source material for each line by its `atf` feature.
As this does not include the line number, we grab that from the `lnno` feature.

In [5]:
for ln in lines:
    print(F.lnno.v(ln), F.atf.v(ln))

2:1' 1'. [x x x] x lu#-u₂# ina KAŠ.SAG [x x x x x x x]
2:2' 2'. [DIŠ NA SA]G.KI.DAB.BA TUKU.TUKU [x x x x]
2:3' 3'. [x x x x x UGU-š]u₂ KUM₂ [x x]
2:4' 4'. [x x x x x x x x Š]U₂? L[UGAL? x x x x]


Note that the document starts with an empty line, which is not ATF-numbered.
Here the feature `lnno` contains `!` followed by the logical line number on the face.

# Text formats

The TF API supports *text formats*. Text formats make selections and apply templates and styles based
on the analysed features of the text. For example: a text-format may ignore flags or clusters, or
format numerals in special ways.

Text formats are not baked into TF, but they are defined in the feature `otext` of the corpus.

Moreover, for this corpus a TF app has been build that defines additional text-formats.

Whereas the formats defined in `otext` are strictly plain text formats, the formats
defined in the app are able to use typographic styles to shape the text, such as bold, italic, colors, etc.

Here is the list of all formats.

In [6]:
T.formats

{'text-orig-full': 'sign',
 'text-orig-plain': 'sign',
 'layout-orig-full': 'sign',
 'layout-orig-plain': 'sign'}

## Plain text formats

The formats whose names start with `text-` are the plain text formats.

### `text-orig-full`

This format is really close to the ATF. It contains all original information.

This is the default format. We do not have to specify it.

In [7]:
for ln in lines:
    print(F.lnno.v(ln), T.text(ln))

2:1' [x x x] x lu#-u₂# ina KAŠ.SAG [x x x x x x x] 
2:2' [DIŠ NA ] SAG#.KI.DAB.BA TUKU.TUKU [x x x x] 
2:3' [x x x x x UGU-] šu₂# KUM₂ [x x] 
2:4' [x x x x x x x x ] ŠU₂?# LUGAL?# [x x x x] 


The `A.plain()` function focuses on the *contents*, and instead of the line number, it gives a full specification
of the location, linked to the online source on CDLI.

In [8]:
for ln in lines:
    A.plain(ln)

# Links to all tablets

We produce a links to CDLI for all tablets in the corpus

In [10]:
for doc in F.otype.s("document"):
    A.plain(doc)

### `text-orig-plain`

This is a somewhat reduced format. It omits all flags and bracketing constructs.

In [9]:
for ln in lines:
    A.plain(ln, fmt="text-orig-plain")

## Styled text formats

The formats whose names start with `layout-` are the styled text formats.

### `layout-orig-full`

This format looks like `text-orig-full`, but now we re-introduce the flags and clusters by specific
layout devices.

See below for detailed examples.

In [10]:
for ln in lines:
    A.plain(ln, fmt="layout-orig-full")

### `layout-orig-plain`

This format looks like `text-orig-plain`, but now we re-introduce the flags and clusters by specific
layout devices.

See below for detailed examples.

In [11]:
for ln in lines:
    A.plain(ln, fmt="layout-orig-plain")

Here is the text of the face in each of the plain text formats, i.e. no additional HTML formatting is applied.

# Pretty

The ultimate of graphical display is by means of the `pretty()` function.

This display is less useful for reading, but instead optimized for showing all information that you might
wish for.

It shows a base representation according to a text format of your choice
(here we choose `layout-orig-full`), and it shows the values
of a standard set of features.

In [12]:
w = F.otype.s("word")[10]
F.atf.v(w)

'hur.ri.im#'

In [13]:
A.pretty(w)

In [14]:
A.pretty(w, fmt="layout-orig-full", withNodes=True)

By default, pretty displays descend to the word level, but you can also descend to the sign level:

In [15]:
A.pretty(w, baseTypes="sign")

In [16]:
A.pretty(w, fmt="layout-orig-full", baseTypes="sign", withNodes=True)

Later on, in the [search](search.ipynb) tutorial we see that `pretty()` can also display other features,
even features that you or other people have created and added later.

Here we call for the feature `atf`, which shows the original atf for the sign in question
excluding the bracketing characters.

Consult the
[feature documentation](https://github.com/Nino-cunei/oldbabylonian/blob/master/docs//transcription.md)
to see what information is stored in all the features.

We show it with node numbers, but you could leave them out in an obvious way.

In [17]:
A.pretty(f, extraFeatures="atf", fmt="layout-orig-full", withNodes=True)

We do not see much, because the default condense type is `line`, and a `document` is bigger than that.
Objects bigger than de condense type will be abbreviated to a label that indicates their identity,
not their contents.

But we can override this by adding `full=True`.

See also the documentation on [`pretty`](https://annotation.github.io/text-fabric/tf/advanced/display.html#tf.advanced.display.pretty).

In [18]:
A.pretty(f, extraFeatures="atf", fmt="layout-orig-plain", withNodes=True, full=True)

# Layout formats: the details

We give detailed examples of how the material is styled in the `layout-` formats.

We show the representation of all kinds of signs and also what the influence of
clustering and flags are.

Here are the design principles:

* all flags `# ? ! *` cause the preceding sign to be in bold
* damage `#` and missing `[ ]` text is blurry and in grey
* questioned `?` and uncertain `( )` text is in italics
* remarkable `!` and supplied `< >` text is overlined, supplied text is in blue
* excised `<< >>` text has a strike-through and is in red
* collated `*` text is underlined

Just a quick overview of the sign types:

In [19]:
F.type.freqList("sign")

(('grapheme', 20433),
 ('reading', 20357),
 ('joiner', 18957),
 ('mark', 12454),
 ('unknown', 4720),
 ('ellipsis', 1236),
 ('numeral', 958),
 ('lang', 186),
 ('wdiv', 92),
 ('empty', 69),
 ('erasure', 26))

# Styled display of ATF text

In [20]:
lines = (
    (("P365742", "reverse", "1:15"), ("cluster: determinative", [9, 10])),
    (("P403381", "obverse", "1':3'"), ("cluster: missing", [2, 3, 4, 5])),
    (("P394104", "obverse", "1:14"), ("cluster: uncertain", [7])),
    (("P285136", "obverse", "6'"), ("cluster: supplied", [11, 12])),
    (("P393782", "obverse", "2:1"), ("cluster: excised", [29, 30])),
    (("P285109", "reverse", "2:1'"), ("flag: damage", [7, 9])),
    (("P285109", "reverse", "2:4'"), ("flag: question", [11, 12])),
    (("P403291", "obverse", "4'"), ("flag: remarkable", [4])),
    (("P394520", "obverse", "2:37'"), ("flag: collated", [28])),
)

In [21]:
for (line, (desc, positions)) in lines:
    ln = T.nodeFromSection(line)
    A.dm("---\n# {}\n\nLocation: {} {}:{}".format(desc, *line))
    s = L.d(ln, otype="sign")[0]
    
    highlights = {s + p - 1 for p in positions}
    print(F.lnno.v(ln), T.text(ln))
    A.plain(ln, fmt="layout-orig-full", highlights=highlights)
    A.pretty(
        ln,
        extraFeatures="tr@en",
        fmt="layout-orig-full",
        baseTypes="sign",
        highlights=highlights,
        withNodes=True,
    )

---
# cluster: determinative

Location: P365742 reverse:1:15

1:15 [ana KI ] MIN# šimGUR₂.GUR₂ šimLI šim.dNIN.URTA NUMUN u₂AB₂.DUH KA A.AB.BA šimŠEŠ DIŠ-niš SUD₂ ina KAŠ SILA₁₁-aš SAR-ab KI MIN 


---
# cluster: missing

Location: P403381 obverse:1':3'

1':3' [... NU pa]-tan# NAG 


---
# cluster: uncertain

Location: P394104 obverse:1:14

...

2:4' [x x x x x x x x ] ŠU₂?# LUGAL?# [x x x x] 


---
# flag: remarkable

Location: P403291 obverse:4'

4' [...] giš!EREN TAG#[-at x x x x x x] 


---
# flag: collated

Location: P394520 obverse:2:37'

2:37' [IGI.MIN-šu₂ DUL-ma u₂-kal IGI.] MIN#-šu₂ ta-kar-ma UD.9.KAM* an-na-a DU₃.MEŠ 


---

All chapters:

* **[start](start.ipynb)** introduction to computing with your corpus
* **display** become an expert in creating pretty displays of your text structures
* **[search](search.ipynb)** turbo charge your hand-coding with search templates
* **[exportExcel](exportExcel.ipynb)** make tailor-made spreadsheets out of your results
* **[similarLines](similarLines.ipynb)** spot the similarities between lines

---

See the [cookbook](cookbook) for recipes for small, concrete tasks.

CC-BY Dirk Roorda